<h1 style="text-align:center; color:#4CAF50;">Driver Drowsiness Detection Using Eye Aspect Ratio (EAR)</h1>

<p style="font-size:16px; line-height:1.6; text-align:justify;">
This project implements a <strong>driver drowsiness detection system</strong> using advanced computer vision techniques and facial landmarks. By analyzing the <em>Eye Aspect Ratio (EAR)</em>, the system detects prolonged eye closure, which could indicate driver fatigue.
</p>

<h2 style="color:#2196F3;">Key Features</h2>
<ul style="font-size:15px;">
  <li>Uses <strong>Dlib's pre-trained model</strong> for facial landmark detection.</li>
  <li>Calculates EAR to determine eye openness or closure.</li>
  <li>Implements a <strong>sliding window mechanism</strong> to track eye states over time.</li>
  <li>Alerts the user upon detecting drowsiness patterns.</li>
</ul>

<h2 style="color:#FF5722;">Technologies Used</h2>
<ul style="font-size:15px;">
  <li><strong>OpenCV</strong> for video frame processing.</li>
  <li><strong>Dlib</strong> for face and eye landmark detection.</li>
  <li><strong>Python</strong> for computational logic and integration.</li>
</ul>

<p style="font-size:16px; line-height:1.6; text-align:center; color:#757575;">
Run the code block below to initialize the system and start detecting driver drowsiness.
</p>


In [ ]:
# Our Setup, Import Libaries, Create our Imshow Function
import cv2
import dlib
import numpy as np

# Importing Land-mark Model
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d shape_predictor_68_face_landmarks.dat.bz2

# Configuration and Constants
EYE_AR_THRESH = 0.22  # Threshold for Eye Aspect Ratio (EAR) to determine eye closure
SLIDING_WINDOW_SIZE = 100  # Number of frames considered in the sliding window
CLOSED_THRESHOLD = 0.4  # 40% of frames with closed eyes triggers an alert
SLEEP_THRESHOLD=0.8
VIDEO_PATH = 'Video.mp4'  # Path to the input video file
LANDMARKS_MODEL = "shape_predictor_68_face_landmarks.dat"  # Path to dlib's pre-trained landmarks model

# Eye Aspect Ratio (EAR) Calculation
def calculate_eye_aspect_ratio(eye_points):
    """
    Calculate the Eye Aspect Ratio (EAR) to determine if an eye is open or closed.
    EAR is based on the distances between horizontal and vertical eye landmarks.
    """
    # Vertical distances
    A = np.linalg.norm(np.array(eye_points[1]) - np.array(eye_points[5]))
    B = np.linalg.norm(np.array(eye_points[2]) - np.array(eye_points[4]))
    # Horizontal distance
    C = np.linalg.norm(np.array(eye_points[0]) - np.array(eye_points[3]))
    # EAR formula
    return (A + B) / (2.0 * C)

# Process detected faces to compute EAR for both eyes
def process_faces(gray_frame, faces, shape_predictor):
    """
    For each detected face, extract the landmarks and compute EAR for both eyes.
    Returns a list of EAR tuples (left_ear, right_ear) for all detected faces.
    """
    results = []
    for face in faces:
        landmarks = shape_predictor(gray_frame, face)
        # Get coordinates for the left and right eyes
        left_eye_points = [(landmarks.part(i).x, landmarks.part(i).y) for i in range(36, 42)]
        right_eye_points = [(landmarks.part(i).x, landmarks.part(i).y) for i in range(42, 48)]
        # Calculate EAR for both eyes
        left_ear = calculate_eye_aspect_ratio(left_eye_points)
        right_ear = calculate_eye_aspect_ratio(right_eye_points)
        results.append((left_ear, right_ear))
    return results

# Initialize Dlib Models
try:
    face_detector = dlib.get_frontal_face_detector()
    shape_predictor = dlib.shape_predictor(LANDMARKS_MODEL)
except Exception as e:
    print(f"Error: Unable to load the landmarks model ({LANDMARKS_MODEL}). Please verify the file path.")
    print(e)
    exit()

# Open Video File
cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    print(f"Error: Could not open video file ({VIDEO_PATH}). Please verify the file path.")
    exit()

# Variables for Tracking
closed_frames = []  # Sliding window to track frames where eyes are closed
frame_count = 1  # Frame counter within each second
sec_count = 1  # Second counter for video playback

# Process video frame-by-frame
while True:
    ret, frame = cap.read()
    if not ret:
        break  # End of video

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Detect faces in the current frame
    faces = face_detector(gray)
    both_eyes_closed = False  # Flag to track if both eyes are closed

    # Process each detected face
    for left_ear, right_ear in process_faces(gray, faces, shape_predictor):
        # Determine the state of each eye based on EAR threshold
        left_eye_closed = left_ear < EYE_AR_THRESH
        right_eye_closed = right_ear < EYE_AR_THRESH
        both_eyes_closed = left_eye_closed and right_eye_closed
        print("")

        # Logging based on eye states
        if both_eyes_closed:
            print(f"{sec_count}, Status: Both eyes closed.", end=" ")
        elif not left_eye_closed and not right_eye_closed:
            print(f"{sec_count}, Status: Both eyes open.", end=" ")
        elif left_eye_closed and not right_eye_closed:
            print(f"{sec_count}, Status: Left eye closed, right eye open.", end=" ")
        elif not left_eye_closed and right_eye_closed:
            print(f"{sec_count}, Status: Left eye open, right eye closed.", end=" ")

    # Update the sliding window with the current frame's eye state
    closed_frames.append(both_eyes_closed)
    if len(closed_frames) > SLIDING_WINDOW_SIZE:
        closed_frames.pop(0)

    # Check for drowsiness based on sliding window
    if len(closed_frames) == SLIDING_WINDOW_SIZE:
        closed_ratio = sum(closed_frames) / SLIDING_WINDOW_SIZE
        if closed_ratio >= CLOSED_THRESHOLD and closed_ratio<=SLEEP_THRESHOLD:
            print(f"ALERT: High drowsiness detected!", end=" ")
        elif closed_ratio > SLEEP_THRESHOLD:
            print(f"ALERT: Driver Sleeping", end=" ")
        else:
            print(f"Driver Awake", end=" ")
        print(f"{int(closed_ratio*100)}% Drowsiness", end=" ")

    # Increment frame and second counters
    frame_count += 1
    if frame_count > 30:  # Assuming 30 FPS
        frame_count = 1
        sec_count += 1

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources and clean up
cap.release()
cv2.destroyAllWindows()
